In [1]:
import scanpy as sc

In [2]:
adata = sc.read("../../data/adamson.h5ad")

# print(adata.obs["UMI count"].head())
# print(adata.var)

In [3]:
adata

AnnData object with n_obs × n_vars = 15006 × 32738
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [4]:
# Feature selection for highly variable genes, feel free to modify parameters

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=2000, layer="counts")

In [5]:
adata = adata[:,adata.var["highly_variable"]]

In [6]:
adata

View of AnnData object with n_obs × n_vars = 15006 × 2000
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'n_genes'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'

In [7]:
adata = adata[~adata.obs["perturbation"].isna() & (adata.obs["perturbation"] != "*")].copy()
adata.obs["perturbation"].unique()

['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF6_PERK_IRE1_pMJ158', 'ATF6_PERK_pMJ150', 'ATF6_only_pMJ145', ..., 'ATF4_pBA576', 'SNAI1_pDS266', 'XBP1_pBA579', 'Gal4-4(mod)_pBA582', 'YIPF5_pDS001']
Length: 19
Categories (19, object): ['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF4_pBA576', 'ATF6_IRE1_pMJ152', ..., 'SNAI1_pDS266', 'XBP1_pBA578', 'XBP1_pBA579', 'YIPF5_pDS001']

In [19]:
%run ../utils.py

In [25]:
split = {"train": 0.7, "val": 0.15, "test": 0.15}
assign_splits(adata, split)
adata.obs["split"].value_counts()


Splitting 19 perts: perts_train=['PERK_only_pMJ146', 'Gal4-4(mod)_pBA582', 'PSMD12_pDS009', 'IRE1_only_pMJ148', 'SNAI1_pDS266', 'YIPF5_pDS001', 'IER3IP1_pDS003', '3x_neg_ctrl_pMJ144-2', 'PERK_IRE1_pMJ154', 'ATF6_PERK_IRE1_pMJ158', 'ATF6_IRE1_pMJ152', 'XBP1_pBA579', 'ATF6_only_pMJ145'], perts_val=['ATF6_PERK_pMJ150', 'ATF4_pBA576', 'PSMA1_pDS007'], perts_test=['XBP1_pBA578', '3x_neg_ctrl_pMJ144-1', 'C7orf26_pDS004']


split
train    11370
test      1719
val       1608
Name: count, dtype: int64